<a href="https://colab.research.google.com/github/ramzigoessing/retail_demand_analysis/blob/main/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import_libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import io

# download_of_data

Here we generate direct download links for our data stored on google drive and load each csv file into a pandas dataframe with descriptive variable names.

In [2]:
def make_drive_url(file_id):
    return f"https://drive.google.com/uc?id={file_id}"

def load_csv_from_url(url):
    response = requests.get(url)
    response.raise_for_status()
    return pd.read_csv(io.StringIO(response.text))

file_ids = {
    "holiday_events": "1RMjSuqHXHTwAw_PGD5XVjhA3agaAGHDH",
    "items": "1ogMRixVhNY6XOJtIRtkRllyOyzw1nqya",
    "oil": "1Q59vk2v4WQ-Rpc9t2nqHcsZM3QWGFje_",
    "stores": "1Ei0MUXmNhmOcmrlPad8oklnFEDM95cDi",
    "train": "1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv",
    "transactions": "1PW5LnAEAiL43fI5CRDn_h6pgDG5rtBW_"
}

df_holiday_events = load_csv_from_url(make_drive_url(file_ids["holiday_events"]))
df_items          = load_csv_from_url(make_drive_url(file_ids["items"]))
df_oil            = load_csv_from_url(make_drive_url(file_ids["oil"]))
df_stores         = load_csv_from_url(make_drive_url(file_ids["stores"]))
df_transactions   = load_csv_from_url(make_drive_url(file_ids["transactions"]))


# download_of_df_train

Since `df_train` is our largest dataset, we have to treat it differently due to the limited memory in Google Colab. We use `gdown` to download the file directly from Google Drive and then load it in chunks, filtering only the stores located in the state of *Guayas*. Finally, we combine the filtered chunks and draw a random sample of 300,000 rows to create a manageable `df_train` DataFrame for further analysis.


In [3]:
!pip install -U gdown
import gdown

train_url = make_drive_url(file_ids["train"])

gdown.download(train_url, "train.csv", quiet=False)

store_ids = df_stores[df_stores['state'] == 'Guayas']['store_nbr'].unique()


chunk_size = 10**6
filtered_chunks = []

for chunk in pd.read_csv("train.csv", chunksize=chunk_size):
    chunk_filtered = chunk[chunk['store_nbr'].isin(store_ids)]
    filtered_chunks.append(chunk_filtered)
    del chunk

df_train = pd.concat(filtered_chunks, ignore_index=True)
df_train = df_train.sample(n=300_000).reset_index(drop=True)
del filtered_chunks

Downloading...
From (original): https://drive.google.com/uc?id=1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv
From (redirected): https://drive.google.com/uc?id=1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv&confirm=t&uuid=47f31935-361c-4d11-86a3-b0348ce90df3
To: /content/train.csv
100%|██████████| 5.00G/5.00G [01:03<00:00, 78.9MB/s]
/tmp/ipython-input-219481652.py:16: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("train.csv", chunksize=chunk_size):


# fill_missing_dates_with_zero_sales

Here, we make sure that the `date` column is a proper datetime type and then enforce a complete daily calendar for each `(store_nbr, item_nbr)` combination. For every store–item pair, we reindex the data to a daily frequency and fill missing dates with zeros. We assume that missing dates for a given store–item pair correspond to days with no sales of the corresponding item, which is why it is reasonable to fill them with zero. This gives us a continuous daily time series per product and store, which is important for later time series analysis and forecasting.

In [4]:
df_train['date'] = pd.to_datetime(df_train['date'])

def fill_calendar(group):

    g = group.set_index("date").sort_index()

    g["store_nbr"] = group["store_nbr"].iloc[0]
    g["item_nbr"]  = group["item_nbr"].iloc[0]

    return g.reset_index()

df_train = (df_train.groupby(["store_nbr", "item_nbr"], group_keys=False).apply(fill_calendar))

/tmp/ipython-input-953193142.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train = (df_train.groupby(["store_nbr", "item_nbr"], group_keys=False).apply(fill_calendar))


# cleaning_onpromotion_and_unit_sales


In this step, we clean two important columns in `df_train`. First, we handle missing values in the `onpromotion` column by filling all `NaN` entries with `False` and casting the column to a boolean type. This assumes that missing entries indicate that the item was not on promotion on that day. Second, we replace negative values in the `unit_sales` column with `0` using `max(x, 0)`. Negative sales typically represent product returns, and for our demand modelling and forecasting purposes we treat these as days with no effective sales rather than allowing negative demand values.


In [5]:
df_train['onpromotion'] = df_train['onpromotion'].fillna(False).astype(bool)

df_train['unit_sales'] = df_train['unit_sales'].apply(lambda x: max(x, 0))

# interpolate_missing_oil_prices


In this step, we handle missing values in the `dcoilwtico` column of `df_oil`, which represents the daily oil price. We use linear interpolation to fill in the gaps, assuming that prices change smoothly between known observations. The `interpolate(method='linear', limit_direction='both')` call fills missing values by drawing straight lines between existing data points and also propagates interpolation forward and backward at the edges of the series if necessary. This results in a continuous oil price time series without `NaN` values, which is important for using oil prices as a reliable external feature in our later analysis and modelling. In particular, we will later use this cleaned oil price series to investigate whether changes in the oil price have an impact on total sales.








In [6]:
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].interpolate(method='linear', limit_direction='both')

# filter_df_train_to_top_3_families

In this step, we focus our analysis on the three largest product families in terms of the number of distinct items. We first count how many items belong to each `family` in `df_items` and extract the top three families based on this item count. Using these families, we then collect all corresponding `item_nbr` values and use them to filter `df_train` so that it only contains rows for items belonging to these top-3 families. This reduces the size and complexity of the training data while still keeping a diverse and representative subset of products for our later analysis and modelling.

In [7]:
items_per_family = df_items['family'].value_counts().reset_index()
items_per_family.columns = ['Family', 'Item Count']
top_3_families = items_per_family.head(3)

item_ids = df_items[df_items['family'].isin(top_3_families['Family'].unique())]['item_nbr'].unique()

df_train = df_train[df_train['item_nbr'].isin(item_ids)]

# detect_outliers_with_zscore

In this step, we identify extreme sales values (outliers) within each `(store_nbr, item_nbr)` combination using the Z-score. We first group `df_train` by `store_nbr` and `item_nbr` and, for each group, compute the mean and standard deviation of `unit_sales`. Based on these statistics, we calculate a Z-score for every observation, measuring how many standard deviations each `unit_sales` value is away from the group mean. To avoid division-by-zero issues, we fall back to a standard deviation of `1` if the actual standard deviation is `0`.  

After computing the Z-scores, we flag all rows with a Z-score greater than `5` as outliers and store them in the `outliers` DataFrame. Finally, we create a new boolean column `outliers` in `df_train` that marks whether a row’s `id` is part of the detected outliers. This allows us to later analyse, filter, or treat these extreme values separately in our modelling pipeline.

In [8]:
def calculate_store_item_zscore(group):
    mean_sales = group['unit_sales'].mean()
    std_sales = group['unit_sales'].std()
    group['z_score'] = (group['unit_sales'] - mean_sales) / (std_sales if std_sales != 0 else 1)
    return group

df_train_grouped = df_train.groupby(['store_nbr', 'item_nbr']).apply(calculate_store_item_zscore)
df_train_grouped.reset_index(drop=True, inplace=True)

outliers = df_train_grouped[df_train_grouped['z_score'] > 5]

print(f"Number of outliers detected: {len(outliers)}")

df_train['outliers'] = df_train['id'].isin(outliers['id'])

/tmp/ipython-input-1760154795.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train_grouped = df_train.groupby(['store_nbr', 'item_nbr']).apply(calculate_store_item_zscore)


Number of outliers detected: 120401


# extract_date_features_for_modelling

In this step, we extract several basic date-related features from the `date` column in `df_train`. Specifically, we create separate columns for the `year`, `month`, and `day`, as well as `day_of_week`, where the day of the week is represented as an integer (Monday = 0, Sunday = 6). These features make calendar information explicit and help our models learn patterns related to seasonality, monthly trends, and weekly effects in the sales data.

In [9]:
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month
df_train['day'] = df_train['date'].dt.day
df_train['day_of_week'] = df_train['date'].dt.dayofweek

# add_holiday_information_to_df_train

In this step, we enrich `df_train` with information about holidays and special events. First, we convert the `date` column in `df_holiday_events` to a proper datetime type so that it is compatible with the `date` column in `df_train`. We then perform a left merge on `date`, bringing the `type` of each holiday or event into `df_train` for all matching dates. Any missing values in this column are filled with `"Regular_Day"`, indicating that there is no special event or holiday on that date. Finally, we rename the `type` column to `holiday` to make its meaning clearer and more explicit in the context of our modelling pipeline. This results in a categorical `holiday` feature that distinguishes between regular days and various types of special days, which can help capture holiday-related effects on sales.

In [10]:
df_holiday_events['date'] = pd.to_datetime(df_holiday_events['date'])
df_train = pd.merge(df_train, df_holiday_events[['date', 'type']], on='date', how='left')
df_train['type'] = df_train['type'].fillna('Regular_Day')
df_train.rename(columns={'type': 'holiday'}, inplace=True)

# add_perishable_flag_to_df_train

In this step, we enrich `df_train` with additional item-level information from `df_items`. We perform a left merge on the `item_nbr` column so that each row in `df_train` receives the corresponding item attributes (such as family or perishability) from `df_items`. After the merge, we cast the `perishable` column to a boolean type, making it explicit whether an item is perishable (`True`) or non-perishable (`False`). This feature is important because perishable items often exhibit different demand patterns and lifecycle behaviour compared to non-perishable products, which can influence our modelling and forecasting results.

In [ ]:
df_train = pd.merge(df_train, df_items, on='item_nbr', how='left')
df_train['perishable'] = df_train['perishable'].astype(bool)

# add_7day_rolling_average_of_unit_sales

In this step, we create a 7-day rolling average of `unit_sales` for each `(item_nbr, store_nbr)` combination. First, we sort `df_train` by `item_nbr`, `store_nbr`, and `date` to ensure that each time series is in correct chronological order. We then group the data by `item_nbr` and `store_nbr` and apply a rolling window of 7 days on the `unit_sales` column, using `min_periods=1` so that the first few days still receive an average based on the available history. The result is stored in a new feature called `unit_sales_7d_avg`, which captures short-term demand trends and smooths out daily fluctuations. This smoothed signal can be very helpful for models that benefit from recent demand dynamics without being overly sensitive to single-day spikes or drops.

In [13]:
df_train = df_train.sort_values(["item_nbr", "store_nbr", "date"]).reset_index(drop=True)
df_train["unit_sales_7d_avg"] = (
    df_train
    .groupby(["item_nbr", "store_nbr"])["unit_sales"]
    .transform(lambda s: s.rolling(window=7, min_periods=1).mean())
)

In [14]:
df_train

,date,id,store_nbr,item_nbr,unit_sales,onpromotion,outliers,year,month,day,day_of_week,holiday,family,class,perishable,unit_sales_7d_avg
0,2013-02-05,1402408,24,96995,1.0,False,True,2013,2,5,1,Regular_Day,GROCERY I,1093,False,1.000000
1,2013-02-06,0,24,96995,0.0,False,False,2013,2,6,2,Regular_Day,GROCERY I,1093,False,0.500000
2,2013-02-07,0,24,96995,0.0,False,False,2013,2,7,3,Regular_Day,GROCERY I,1093,False,0.333333
3,2013-02-08,0,24,96995,0.0,False,False,2013,2,8,4,Regular_Day,GROCERY I,1093,False,0.250000
4,2013-02-09,0,24,96995,0.0,False,False,2013,2,9,5,Regular_Day,GROCERY I,1093,False,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17843942,2017-08-12,125134313,29,2113343,1.0,False,False,2017,8,12,5,Regular_Day,BEVERAGES,1114,False,1.000000
17843943,2017-07-21,122819909,29,2113914,12.0,True,False,2017,7,21,4,Regular_Day,CLEANING,3040,False,12.000000
17843944,2017-08-14,125354524,36,2116416,1.0,False,False,2017,8,14,0,Regular_Day,GROCERY I,1060,False,1.000000
17843945,2017-08-10,124924419,27,2122188,3.0,False,False,2017,8,10,3,Holiday,GROCERY I,1084,False,3.000000


# save_clean_df_train_to_google_drive

In this step, we save our cleaned `df_train` DataFrame to Google Drive so that we can easily reuse it later without having to repeat all preprocessing steps. First, we mount Google Drive in the Colab environment using `drive.mount('/content/drive')`, which requires a one-time authorization click. Next, we define a `save_path` inside our Drive where the file should be stored. Finally, we use `df_train.to_pickle(save_path)` to save the DataFrame in pickle format, which preserves data types and is efficient to load back into memory. This allows us to quickly reload the preprocessed dataset in future notebooks or sessions by simply reading the pickle file from the same path.

In [15]:
from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/time_series_course/df_train_clean.pkl" # example of path

df_train.to_pickle(save_path)
print(f"Saved to: {save_path}")

Mounted at /content/drive
Saved to: /content/drive/MyDrive/time_series_course/df_train_clean.pkl
